In [ ]:
cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.

In [ ]:
hindi_sen = open('IITB.en-hi.hi').read().split("\n")[:-1]
eng_sen = open('IITB.en-hi.en').read().split("\n")[:-1]


In [ ]:
hindi_sen[:10]

['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें',
 'एक्सेर्साइसर पहुंचनीयता अन्वेषक',
 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका',
 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका',
 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है',
 'अवधि को हाइलाइट रकें',
 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि',
 'सीमांत (बोर्डर) के रंग को हाइलाइट करें',
 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। ',
 'भराई के रंग को हाइलाइट करें']

In [ ]:
eng_sen[:10]

['Give your application an accessibility workout',
 'Accerciser Accessibility Explorer',
 'The default plugin layout for the bottom panel',
 'The default plugin layout for the top panel',
 'A list of plugins that are disabled by default',
 'Highlight duration',
 'The duration of the highlight box when selecting accessible nodes',
 'Highlight border color',
 'The color and opacity of the highlight border.',
 'Highlight fill color']

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [ ]:
input_text = eng_sen[:10000]
target_texts = hindi_sen[:10000]
len(input_text)

10000

In [ ]:
target_text = []

for i in target_texts:
  target_text.append( "\t" + i + "\n")


In [ ]:
target_text[0]

'\tअपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें\n'

In [ ]:
print(len(input_text))
print(len(target_text))

10000
10000


In [ ]:
# Vectorize the data.
input_characters = set()
target_characters = set()
for char in input_text:
  for chars in char:
    if chars not in input_characters:
      input_characters.add(chars)
for char in target_text:
  for chars in char:
    if chars not in target_characters:
      target_characters.add(chars)

In [ ]:
input_characters

{' ',
 '!',
 '"',
 '#',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '~',
 '©',
 '“',
 '”',
 '…'}

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_text])
max_decoder_seq_length = max([len(txt) for txt in target_text])

print("Number of samples:", len(input_text))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

Number of samples: 10000
Number of unique input tokens: 93
Number of unique output tokens: 154
Max sequence length for inputs: 233
Max sequence length for outputs: 346


In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '#': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '<': 27,
 '=': 28,
 '>': 29,
 '?': 30,
 'A': 31,
 'B': 32,
 'C': 33,
 'D': 34,
 'E': 35,
 'F': 36,
 'G': 37,
 'H': 38,
 'I': 39,
 'J': 40,
 'K': 41,
 'L': 42,
 'M': 43,
 'N': 44,
 'O': 45,
 'P': 46,
 'Q': 47,
 'R': 48,
 'S': 49,
 'T': 50,
 'U': 51,
 'V': 52,
 'W': 53,
 'X': 54,
 'Y': 55,
 'Z': 56,
 '[': 57,
 '\\': 58,
 ']': 59,
 '_': 60,
 '`': 61,
 'a': 62,
 'b': 63,
 'c': 64,
 'd': 65,
 'e': 66,
 'f': 67,
 'g': 68,
 'h': 69,
 'i': 70,
 'j': 71,
 'k': 72,
 'l': 73,
 'm': 74,
 'n': 75,
 'o': 76,
 'p': 77,
 'q': 78,
 'r': 79,
 's': 80,
 't': 81,
 'u': 82,
 'v': 83,
 'w': 84,
 'x': 85,
 'y': 86,
 'z': 87,
 '~': 88,
 '©': 89,
 '“': 90,
 '”': 91,
 '…': 92}

In [ ]:
encoder_input_data = np.zeros(
    (len(input_text), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_text), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_text), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_texts, target_texts) in enumerate(zip(input_text, target_text)):
    for t, char in enumerate(input_texts):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_texts):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 19s 95ms/step - loss: 0.7462 - accuracy: 0.8970 - val_loss: 0.1871 - val_accuracy: 0.9639
Epoch 2/100
125/125 [==============================] - 11s 89ms/step - loss: 0.3045 - accuracy: 0.9404 - val_loss: 0.1567 - val_accuracy: 0.9650
Epoch 3/100
125/125 [==============================] - 11s 87ms/step - loss: 0.3141 - accuracy: 0.9417 - val_loss: 0.1456 - val_accuracy: 0.9665
Epoch 4/100
125/125 [==============================] - 11s 87ms/step - loss: 0.2133 - accuracy: 0.9467 - val_loss: 0.1289 - val_accuracy: 0.9681
Epoch 5/100
125/125 [==============================] - 11s 87ms/step - loss: 0.1795 - accuracy: 0.9549 - val_loss: 0.1226 - val_accuracy: 0.9686
Epoch 6/100
125/125 [==============================] - 11s 86ms/step - loss: 0.1545 - accuracy: 0.9602 - val_loss: 0.1164 - val_accuracy: 0.9702
Epoch 7/100
125/125 [==============================] - 11s 87ms/step - loss: 0.1409 - accuracy: 0.9630 - val_loss: 0.1121 - val_ac

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

In [ ]:
model.input

[<KerasTensor: shape=(None, None, 93) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, None, 154) dtype=float32 (created by layer 'input_2')>]

In [ ]:
model.layers

In [ ]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_5")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_6")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [ ]:
decoder_lstm

In [ ]:
decoder_inputs.shape

TensorShape([None, None, 154])

In [ ]:
decoder_states_inputs

[<KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_5')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_6')>]

In [ ]:

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_text[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Give your application an accessibility workout
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: Accerciser Accessibility Explorer
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: The default plugin layout for the bottom panel
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: The default plugin layout for the top panel
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: A list of plugins that are disabled by default
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: Highlight duration
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: The duration of the highlight box when selecting accessible nodes
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: Highlight border color
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: The color and opacity of the highlight border.
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: Highlight fill color
Decoded sentence: प्रोजेक्ट बंद करें

-
Input sentence: The color and opacity of th